# Chapter5-5

## 多肢選択式質問応答

5卓のクイズ問題を使った常識推論のタスク。<br>
問題文と選択しのペアを作り、それぞれの妥当性を計算し、一番高いスコアの選択肢を解答として出力する

## ライブラリのインストール

In [1]:
!pip install transformers[ja,torch] datasets matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1

## データセットを読み込む

In [2]:
from pprint import pprint
from datasets import load_dataset

# データセットの読み込み
train_dataset = load_dataset(
    "llm-book/JGLUE", name="JCommonsenseQA", split="train"
)

valid_dataset = load_dataset(
    "llm-book/JGLUE", name="JCommonsenseQA", split="validation"
)

# データを表示
pprint(train_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.w

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'choice0': '世界',
 'choice1': '写真集',
 'choice2': '絵本',
 'choice3': '論文',
 'choice4': '図鑑',
 'label': 2,
 'q_id': 0,
 'question': '主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？'}


In [3]:
from transformers import BatchEncoding

def preprocess_multiple_choise(example: dict[str, str]) -> BatchEncoding:
  """ 多肢選択式質問応答の事例をIDに変換 """

  # 選択肢の数を"choice"から始まるキーの数として算出
  num_choices = sum(key.startswith("choice") for key in example.keys())

  # 質問と選択しを連結してトークナイザーに渡す
  choice_list = [example[f"choice{i}"] for i in range(num_choices)]
  repeated_question_list = [example["question"]] * num_choices
  encoded_example = tokenizer(
      repeated_question_list, choice_list, max_length=64
  )

  # ラベルが入力に含まれている場合に、出力にも追加する
  if "label" in example:
    encoded_example["labels"] = example["label"]

  return encoded_example

In [4]:
from transformers import AutoTokenizer

# トークナイザーの読み込み
transformers_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(transformers_model_name)

example = train_dataset[0]

encoded_example = preprocess_multiple_choise(example)
for choice in range(5):
  # IDから元の文字列の復元
  print(tokenizer.decode(encoded_example["input_ids"][choice]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[CLS] 主に 子ども 向け の もの で 、 イラスト の つい た 物語 が 書か れ て いる もの は どれ? [SEP] 世界 [SEP]
[CLS] 主に 子ども 向け の もの で 、 イラスト の つい た 物語 が 書か れ て いる もの は どれ? [SEP] 写真 集 [SEP]
[CLS] 主に 子ども 向け の もの で 、 イラスト の つい た 物語 が 書か れ て いる もの は どれ? [SEP] 絵本 [SEP]
[CLS] 主に 子ども 向け の もの で 、 イラスト の つい た 物語 が 書か れ て いる もの は どれ? [SEP] 論文 [SEP]
[CLS] 主に 子ども 向け の もの で 、 イラスト の つい た 物語 が 書か れ て いる もの は どれ? [SEP] 図鑑 [SEP]


## ミニバッチの構築

In [5]:
import torch
from transformers import BatchEncoding

def collate_fn_multiple_choice(features: list[BatchEncoding]) -> dict[str, torch.Tensor]:
  """ 選択肢式質問応答の入力からミニバッチを構築 """

  # preprocess_multiple_choice関数に合わせてラベル名を"labels"とする
  label_name = "labels"

  batch_size = len(features)
  num_choices = len(features[0]["input_ids"])

  # 選択肢ごとの入力をひとつのlistにまとめる
  flat_features = []
  for feature in features:
    flat_features += [
        {k: v[i] for k, v in feature.items() if k != label_name}
        for i in range(num_choices)
    ]

  # 選択肢ごとの入力についてパディングを行う
  flat_batch = tokenizer.pad(flat_features, return_tensors="pt")

  # もとのバッチごとに選択肢ごとの入力をまとめる
  # (バッチサイズ * 選択肢数, 最大系列長) の形をしたTensorを、
  # (バッチサイズ、選択肢数、最大系列長)に変換する
  batch = {
      k: v.view(batch_size, num_choices, -1)
      for k, v in flat_batch.items()
  }

  # ラベルに入力が含まれているとき、バッチにまとめてTensorに変換
  if label_name in features[0]:
    labels = [feature[label_name] for feature in features]
    batch["labels"] = torch.tensor(labels, dtype=torch.int64)

  return batch

In [6]:
batch_size = 4
encoded_examples = [
    preprocess_multiple_choise(train_dataset[i])
    for i in range(batch_size)
]

batch = collate_fn_multiple_choice(encoded_examples)
pprint({name: tensor.size() for name, tensor in batch.items()})

{'attention_mask': torch.Size([4, 5, 40]),
 'input_ids': torch.Size([4, 5, 40]),
 'labels': torch.Size([4]),
 'token_type_ids': torch.Size([4, 5, 40])}


選択肢ごとのテキストからミニバッチが構築されている。<br>
バッチサイズ $\times$ 選択肢数 $\times$ 最大系列長 の`Tensor`が作られる

## モデルの読み込み

In [7]:
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained(
    transformers_model_name,
    num_labels=train_dataset.features["label"].num_classes
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# ミニバッチをモデルに入力
model(**batch)

MultipleChoiceModelOutput(loss=tensor(1.6233, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0226,  0.0595, -0.0720, -0.1338, -0.1835],
        [ 0.1983,  0.2099,  0.1674,  0.3066,  0.1628],
        [ 0.2482,  0.1405,  0.1636,  0.1721,  0.1548],
        [ 0.2529,  0.5137,  0.3532,  0.1814,  0.2888]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

`logits`: ヘッドから出力されている`Tensor`、各選択肢に対するスコアを表す<br>
`loss`: スコアに対して`softmax`を適用して得た予測分布と、正解ラベルから計算される `Cross Entropy`の値

## 学習の実行

In [9]:
# データセットの準備
encoded_train_dataset = train_dataset.map(
    preprocess_multiple_choise,
    remove_columns=train_dataset.column_names
)

encoded_valid_dataset = valid_dataset.map(
    preprocess_multiple_choise,
    remove_columns=valid_dataset.column_names
)

Parameter 'function'=<function preprocess_multiple_choise at 0x79d9093b16c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/8939 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [10]:

# 学習率、バッチサイズ、ハイパーパラメータなどの学習にかかわる設定をする
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Learning_LLM/chapter5/checkpoint", # 結果を保存するフォルダ
    per_device_train_batch_size=32, # 学習時のバッチサイズ
    per_device_eval_batch_size=32, # 評価時のバッチサイズ
    learning_rate=2e-5, # 学習率
    lr_scheduler_type="linear", # 学習率スケジュラーの種類
    warmup_ratio=0.1, # 学習率のウォームアップの長さを指定
    num_train_epochs=3, # エポック数
    save_strategy="epoch", # チェックポイントの保存のタイミング (今回は1エポック毎に行う)
    logging_strategy="epoch", # ロギングのタイミング (1エポック毎)
    evaluation_strategy="epoch", # 検証セットによる評価のタイミング(1エポック毎)
    load_best_model_at_end=True, # 学習後に最良のモデルをロードさせる
    metric_for_best_model="accuracy", # 最良のモデルを決定する際の評価指標
    fp16=True, # 自動混合制度演算を有効にする
)

In [11]:
# モデルの評価方法を定義する
import numpy as np

def compute_accuracy(eval_pred: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
  """ 予測ラベルと正解ラベルから正解率を計算 """

  predictions, labels = eval_pred

  # predictions: 各ラベルについてのスコア
  # 一番スコアの高いインデックスを予測ラベルにする
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": (predictions == labels).mean()}

In [12]:
# 学習を開始する
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=collate_fn_multiple_choice,
    args=training_args,
    compute_metrics=compute_accuracy
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.825100,0.459116,0.830206
2,0.405700,0.448852,0.827525
3,0.289100,0.450804,0.832887


TrainOutput(global_step=840, training_loss=0.5065967559814453, metrics={'train_runtime': 264.6704, 'train_samples_per_second': 101.322, 'train_steps_per_second': 3.174, 'total_flos': 2308796766950610.0, 'train_loss': 0.5065967559814453, 'epoch': 3.0})

In [13]:
# モデルの保存
trainer.save_model("/content/drive/MyDrive/Learning_LLM/chapter5/model")
tokenizer.save_pretrained("/content/drive/MyDrive/Learning_LLM/chapter5/model")

('/content/drive/MyDrive/Learning_LLM/chapter5/model/tokenizer_config.json',
 '/content/drive/MyDrive/Learning_LLM/chapter5/model/special_tokens_map.json',
 '/content/drive/MyDrive/Learning_LLM/chapter5/model/vocab.txt',
 '/content/drive/MyDrive/Learning_LLM/chapter5/model/added_tokens.json')

In [14]:
# 評価の実行
eval_metrics = trainer.evaluate()
pprint(eval_metrics)

{'epoch': 3.0,
 'eval_accuracy': 0.8328865058087578,
 'eval_loss': 0.45080438256263733,
 'eval_runtime': 3.0872,
 'eval_samples_per_second': 362.46,
 'eval_steps_per_second': 11.337}


## モデルの推論

これには`pipeline`での実装はできないため、
モデルとトークナイザから推論のための関数を作成する

In [15]:
from pprint import pprint
from datasets import load_dataset
from transformers import (
    AutoModelForMultipleChoice,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer
)

# モデル、トークナイザの読み込み
model_path = "/content/drive/MyDrive/Learning_LLM/chapter5/model"
model = AutoModelForMultipleChoice.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [24]:
def pipeline_multiple_choice(
    examples: dict[str, str] | list[dict[str, str]]
) -> list[dict[str, str]]:
  """ 多肢選択式質問応答の事例を予測 """

  # 単一のdictの入力が与えられたときにlistに格納する
  if isinstance(examples, dict):
    examples = [examples]

  # 事例をモデルの入力形式に変換
  encoded_examples = [
      preprocess_multiple_choise(e) for e in examples
  ]

  batch = collate_fn_multiple_choice(encoded_examples)

  # モデルが使用するデバイス上にデータを移動する
  batch = {k: v.to(model.device) for k, v in batch.items()}

  # モデルの前向き計算処理
  model_output = model.forward(**batch)

  # モデルの予測を選択肢の文字列に変換
  predicted_ids = model_output.logits.argmax(dim=-1).tolist()
  predictions = [
      {"prediction": e[f"choice{i}"]}
      for e, i in zip(examples, predicted_ids)
  ]

  return predictions

In [25]:
# モデルの動作確認
choices = ["水", "石", "火", "氷", "天然ガス", "石油"]
choices_dict = {f"choice{i}": c for i, c in enumerate(choices)}
example1 = {"question": "液体はどれ？", **choices_dict}
example2 = {"question": "気体はどれ？", **choices_dict}

# 事例1つだけ
print(pipeline_multiple_choice(example1))

# 複数の事例
print(pipeline_multiple_choice([example1, example2]))

[{'prediction': '水'}]
[{'prediction': '水'}, {'prediction': '天然ガス'}]
